<a href="https://colab.research.google.com/github/Ucchwas/ECG-Classification-using-CNN/blob/main/CNN_Multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
import warnings
warnings.filterwarnings('ignore')

In [35]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/mitbih_train_Ihua.csv')

In [36]:
df.drop('1',axis='columns',inplace=True)
#print(df.dtypes)
df.fillna(0, inplace=True)
X = df.drop('Label1',axis='columns')
#Xs = df.drop('Label1',axis='columns')
#X = Xs[df['Label1'] != 1 & 2 & 3]
y = testLabels = df.Label1.astype(np.float32)
#ys = testLabels = df.Label1.astype(np.float32)
#y = ys[df['Label1'] != 1 & 2 & 3]

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2,train_size=0.8, random_state=15, stratify=y)
#print(y_train.value_counts())
#print(y.value_counts())
#print(y.value_counts())
#print(len(X_train.columns))

#from tensorflow_addons import losses
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

In [38]:
from keras.layers import Input, Convolution1D, BatchNormalization, MaxPooling1D, Flatten, Dense
from keras.models import Model

In [39]:
def CNN(X_train, y_train, X_test, y_test, loss, weights):
    im_shape = (X_train.shape[1], 1)
    inputs_cnn = Input(shape=im_shape, name='inputs_cnn')
    conv1_1 = Convolution1D(64, 6, activation='relu', input_shape=im_shape)(inputs_cnn)
    conv1_1 = BatchNormalization()(conv1_1)
    pool1 = MaxPooling1D(pool_size=3, strides=2, padding="same")(conv1_1)
    conv2_1 = Convolution1D(64, 3, activation='relu')(pool1)
    conv2_1 = BatchNormalization()(conv2_1)
    pool2 = MaxPooling1D(pool_size=2, strides=2, padding="same")(conv2_1)
    conv3_1 = Convolution1D(64, 3, activation='relu')(pool2)
    conv3_1 = BatchNormalization()(conv3_1)
    pool3 = MaxPooling1D(pool_size=2, strides=2, padding="same")(conv3_1)
    flatten = Flatten()(pool3)
    dense_end1 = Dense(64, activation='relu')(flatten)
    dense_end2 = Dense(32, activation='relu')(dense_end1)
    main_output = Dense(5, activation='sigmoid', name='main_output')(dense_end2)

    model = Model(inputs=inputs_cnn, outputs=main_output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    if weights == -1:
        model.fit(X_train, y_train, epochs=50)
    else:
        model.fit(X_train, y_train, epochs=50, class_weight=weights)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_train)
    y_preds = (y_preds > 0.5).astype(int)

    print("Classification Report: \n", classification_report(y_train, y_preds))
    return model, y_preds

In [40]:
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.layers import BatchNormalization
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [41]:
from keras.utils import to_categorical

# Convert y_train to one-hot encoding
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [42]:
model,history=CNN(X_train,y_train,X_test,y_test, 'binary_crossentropy', -1)

Epoch 1/50
2189/2189 [==============================] - 88s 39ms/step - loss: 0.0519 - accuracy: 0.9603
Epoch 2/50
2189/2189 [==============================] - 84s 38ms/step - loss: 0.0279 - accuracy: 0.9790
Epoch 3/50
2189/2189 [==============================] - 91s 42ms/step - loss: 0.0217 - accuracy: 0.9836
Epoch 4/50
2189/2189 [==============================] - 84s 38ms/step - loss: 0.0180 - accuracy: 0.9859
Epoch 5/50
2189/2189 [==============================] - 86s 39ms/step - loss: 0.0158 - accuracy: 0.9875
Epoch 6/50
2189/2189 [==============================] - 83s 38ms/step - loss: 0.0133 - accuracy: 0.9895
Epoch 7/50
2189/2189 [==============================] - 85s 39ms/step - loss: 0.0119 - accuracy: 0.9905
Epoch 8/50
2189/2189 [==============================] - 83s 38ms/step - loss: 0.0104 - accuracy: 0.9916
Epoch 9/50
2189/2189 [==============================] - 84s 38ms/step - loss: 0.0094 - accuracy: 0.9921
Epoch 10/50
2189/2189 [==============================] - 84s 38m

In [50]:
model.save("CNN_Multiclass.h5")

In [51]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model("CNN_Multiclass.h5")

In [52]:
# convert to tensorflow lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("CNN_Multiclass.tflite", "wb").write(tflite_model)

482892